In [ ]:
import openpyxl
import os

def copy_cell_styles(source_cell, target_cell):
    """Copy the style attributes from the source cell to the target cell."""
    target_cell.font = openpyxl.styles.Font(name=source_cell.font.name,
                                            size=source_cell.font.size,
                                            bold=source_cell.font.bold,
                                            italic=source_cell.font.italic,
                                            vertAlign=source_cell.font.vertAlign,
                                            underline=source_cell.font.underline,
                                            strike=source_cell.font.strike,
                                            color=source_cell.font.color)
    
    target_cell.fill = openpyxl.styles.PatternFill(fill_type=source_cell.fill.fill_type,
                                                   start_color=source_cell.fill.start_color,
                                                   end_color=source_cell.fill.end_color)
    
    target_cell.border = openpyxl.styles.Border(left=source_cell.border.left,
                                                right=source_cell.border.right,
                                                top=source_cell.border.top,
                                                bottom=source_cell.border.bottom)
    
    target_cell.alignment = openpyxl.styles.Alignment(horizontal=source_cell.alignment.horizontal,
                                                      vertical=source_cell.alignment.vertical,
                                                      text_rotation=source_cell.alignment.text_rotation,
                                                      wrap_text=source_cell.alignment.wrap_text,
                                                      shrink_to_fit=source_cell.alignment.shrink_to_fit,
                                                      indent=source_cell.alignment.indent)
    
    target_cell.number_format = source_cell.number_format
    target_cell.protection = openpyxl.styles.Protection(locked=source_cell.protection.locked,
                                                        hidden=source_cell.protection.hidden)

def copy_dimensions(source_sheet, target_sheet):
    """Copy row heights and column widths from source sheet to target sheet."""
    # Copy column widths
    for col_letter, col_dim in source_sheet.column_dimensions.items():
        target_sheet.column_dimensions[col_letter].width = col_dim.width
    
    # Copy row heights
    for row_idx, row_dim in source_sheet.row_dimensions.items():
        target_sheet.row_dimensions[row_idx].height = row_dim.height

def copy_merged_cells(source_sheet, target_sheet):
    """Copy merged cell ranges from source sheet to target sheet."""
    for merged_range in source_sheet.merged_cells.ranges:
        target_sheet.merge_cells(str(merged_range))

def split_excel(input_file, output_folder, header_row, split_column):
    try:
        # Check if the output folder exists, if not, create it
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        # Load the Excel file
        wb = openpyxl.load_workbook(input_file)
        sheet = wb.active

        # Collect header rows (from the first row up to and including the specified header row)
        header_rows = list(sheet.iter_rows(min_row=1, max_row=header_row))

        # Determine the split column index (0-based)
        split_column_index = split_column - 1

        # Collect unique values from the split column
        split_column_values = []
        for row in sheet.iter_rows(min_row=header_row + 1):
            cell_value = row[split_column_index].value
            if cell_value not in split_column_values:
                split_column_values.append(cell_value)

        # Split and save each subset of data based on unique values in the split column
        for value in split_column_values:
            filename = os.path.join(output_folder, f"split_{value}.xlsx")
            print("Generating file:", filename)

            # Create a new Workbook for each unique value
            wb_new = openpyxl.Workbook()
            sheet_new = wb_new.active
            sheet_new.title = "Sheet1"

            # Copy column widths, row heights, and merged cells from the original sheet
            copy_dimensions(sheet, sheet_new)
            copy_merged_cells(sheet, sheet_new)

            # Write all header rows exactly as specified with styles
            for row_idx, header_row_cells in enumerate(header_rows, start=1):
                for col_idx, source_cell in enumerate(header_row_cells, start=1):
                    target_cell = sheet_new.cell(row=row_idx, column=col_idx, value=source_cell.value)
                    copy_cell_styles(source_cell, target_cell)

            # Write rows that match the split column value, preserving styles
            row_new = header_row + 1  # Start writing from the row after the header rows
            for row in sheet.iter_rows(min_row=header_row + 1):
                if row[split_column_index].value == value:
                    for col_idx, source_cell in enumerate(row, start=1):
                        target_cell = sheet_new.cell(row=row_new, column=col_idx, value=source_cell.value)
                        copy_cell_styles(source_cell, target_cell)
                    row_new += 1

            # Save the split file
            wb_new.save(filename)
            print("File saved:", filename)

        print("All files split successfully.")

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
input_file = r'/Users/gejundeng/Desktop/xxxxx.xlsx'  # Input file path
output_folder = r'/Users/gejundeng/Desktop/拆分文件夹'  # Output folder path
header_row = 2  # Header row (1-based index)
split_column = 2  # Split column (1-based index)

# Call the function
split_excel(input_file, output_folder, header_row, split_column)